# Orbit Fitting Tutorial

This tutorial shows how to fit the visual orbit of a directly imaged planet using `orbitize!`.

In [ ]:
!pip install orbitize

# Create an orbitize data file. 

Follow the example data file [example_astrometry.csv](https://github.com/semaphoreP/ASTRON314-414/blob/main/FinalProject/example_astrometry.csv?plain=1) and make your own data file listing the astrometry you measured from the Imaging Exoplanets Tutorial. 

Each row of the data file contains one epoch of astrometry. The fields are as follows:
  1. `epoch`: this is the date in MJD (units of days). For the 2009 dataset, the MJD is 55137. For the 2021 dataset, the MJD is 59404. 
  2. `body`: 1 is for the first companion. If you end up fitting more than one companion, the next one is 2, followed by 3, etc... (however, I recommend you fit each planet separately to make the computation easier).
  3. `sep`: the separation of the companion in this epoch in mas
  4. `sep_err`: the error on the separation in mas
  5. `pa`: the position angle of the companion in this epoch in degrees
  6. `pa_err`: the error on the position angle in degrees. 

# Run the orbitize fit

In [ ]:
import orbitize
import orbitize.driver

import matplotlib.pyplot as plt
%matplotlib inline

# Set up the system

Look up what should be used for the total mass and parallax of the system by following values used in the literature. The numbers here are totally wrong!!

In [ ]:
myDriver = orbitize.driver.Driver('example_astrometry.csv'.format(orbitize.DATADIR), # path to data file
                                  'OFTI', # name of algorithm for orbit-fitting
                                  1, # number of secondary bodies in system
                                  1.22, # total mass [M_sun]
                                  56.95) # total parallax of system [mas]


# Find orbits

We will look the find 1000 possible orbital solutions. Note that this could take 1-2 minutes. If you try to fit both planets at the same time, it may take much longer. 

In [ ]:
s = myDriver.sampler
orbits = s.run_sampler(1000)

# Plot orbit and range of possible orbital parameters

In [ ]:
my_results = s.results

epochs = my_results.system.data_table['epoch']

orbit_figure = my_results.plot_orbits(
    start_mjd=epochs[0] # Minimum MJD for colorbar (here we choose first data epoch)
)

In [ ]:
sma = np.array([x[s.system.param_idx['sma1']] for x in orbits])
ecc = np.array([x[s.system.param_idx['ecc1']] for x in orbits])
i = np.degrees(np.array([x[s.system.param_idx['inc1']] for x in orbits]))

plt.figure(figsize=(10,3))
plt.subplot(131)
logbins = np.logspace(np.log10(np.min(sma)),np.log10(np.max(sma)),30)
plt.hist(sma, bins=logbins)
plt.gca().set_xscale("log")
plt.xlabel('orbital semi-major axis [AU]')
plt.ylabel('occurrence')

plt.subplot(132)
plt.hist(ecc, bins=30)
plt.xlabel('eccentricity [0,1]')
plt.ylabel('occurrence')

plt.subplot(133)
plt.hist(i, bins=30)
plt.xlabel('inclination angle [deg]')
plt.ylabel('occurrence')

plt.tight_layout()

print("median SMA", np.median(sma))
print("median ecc", np.median(ecc))
print("median Inc", np.median(i))
